In [ ]:
import gradio as gr

def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch()


In [ ]:
gr.themes.builder()


In [ ]:
import gradio as gr
from gradio.themes.base import Base
import time

class Seafoam(Base):
    pass

seafoam = Seafoam()

with gr.Blocks(theme=seafoam) as demo:
    textbox = gr.Textbox(label="Name")
    slider = gr.Slider(label="Count", minimum=0, maximum=100, step=1)
    with gr.Row():
        button = gr.Button("Submit", variant="primary")
        clear = gr.Button("Clear")
    output = gr.Textbox(label="Output")

    def repeat(name, count):
        time.sleep(3)
        return name * count
    
    button.click(repeat, [textbox, slider], output)

demo.launch()


In [ ]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo:
     def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)


demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)


demo.launch()

In [ ]:
import gradio as gr

title = "Multiple Interfaces"

#app 1
def user_greeting(name):
    return "Hi! " + name + " Welcome to your first Gradio application!😎"

#app 2
def user_help(do):
    return "So today we will do " + do + "using Gradio. Great choice!"

#interface 1
app1 =  gr.Interface(fn = user_greeting, inputs="text", outputs="text")
#interface 2

app2 =  gr.Interface(fn = user_help, inputs="text", outputs="text")

demo = gr.TabbedInterface([app1, app2], ["Welcome", "What to do"])

demo.launch()

In [ ]:
import gradio as gr
import numpy as np

In [ ]:
def reverse_audio(audio):
    sr, data = audio
    return (sr, np.flipud(data))

In [ ]:
input_audio = gr.Audio(sources=["microphone"])
demo = gr.Interface(
    fn=reverse_audio,
    inputs=input_audio,
    outputs="audio"
)


In [ ]:
if __name__ == "__main__":
    demo.launch()

Speech to Audio with Gradio ASR and transformers

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np

# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio # Extract sampling rate and audio data
    y = y.astype(np.float32) # Convert audio data to float32
    y /= np.max(np.abs(y)) # Normalize the audio data
    return transcriber({"sampling_rate": sr, "raw": y})["text"] # Transcribe the audio


In [ ]:
demo = gr.Interface(
    transcribe,
    gr.Audio(sources=["microphone"]),
    "text",
)


In [ ]:
demo.launch()

Gradio Interface with ASR returning text to dataframe

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
import pandas as pd

# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    return transcriber({"sampling_rate": sr, "raw": y})["text"]


In [ ]:
#create dataframe from transcription

def transcribe_to_df(audio):
    transcribed_text = transcribe(audio)
    # Create a DataFrame with the transcribed text
    df = pd.DataFrame({"Transcription": [transcribed_text]})
    return df


In [ ]:
demo = gr.Interface(
    transcribe_to_df,
    gr.Audio(sources=["microphone"]),
    outputs="dataframe",
)

In [ ]:
demo.launch()


In [ ]:
df_push = pd.DataFrame(dataframe)

Gradio audio caputre to Assembly AI sentiment analysis

In [ ]:
import gradio as gr

# Placeholder function to stream audio to AssemblyAI
def stream_audio_to_assemblyai(audio_stream):
    # This function should establish a WebSocket connection to AssemblyAI,
    # send the audio stream in real-time, and return the transcription and sentiment analysis results.
    # The actual implementation depends on the AssemblyAI API and WebSocket handling.
    pass

# Gradio interface for live audio input
def process_live_audio(audio_stream):
    # Process the live audio stream
    transcription_and_sentiment = stream_audio_to_assemblyai(audio_stream)
    return transcription_and_sentiment

iface = gr.Interface(
    fn=process_live_audio,
    inputs=gr.Audio(sources=["microphone"], streaming=True),
    outputs=["text", "json"], # Assuming sentiment analysis results are returned in JSON format
    live=True,
)

iface.launch()


Gradio ASR to Assembly AI

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
import requests

# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    return transcriber({"sampling_rate": sr, "raw": y})["text"]

def send_to_assemblyai(text):
    # Replace with your actual AssemblyAI API endpoint and API key
    url = "https://api.assemblyai.com/v2/transcript"
    headers = {
        "Authorization": "6898dad2ca8f4fd2862cfe04fd51e992"
    }
    data = {
        "text": text,
        # Additional parameters as needed
    }
    response = requests.post(url, headers=headers, json=data)
    return response.json()

def process_audio(audio):
    transcribed_text = transcribe(audio)
    assemblyai_response = send_to_assemblyai(transcribed_text)
    return assemblyai_response

iface = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(sources=["microphone"]),
    outputs="json",
)

iface.launch()


In [ ]:
# `pip3 install assemblyai` (macOS)
# `pip install assemblyai` (Windows)

import assemblyai as aai

aai.settings.api_key = "6898dad2ca8f4fd2862cfe04fd51e992"
transcriber = aai.Transcriber()

transcript = transcriber.transcribe("https://storage.googleapis.com/aai-web-samples/news.mp4")
# transcript = transcriber.transcribe("./my-local-audio-file.wav")

print(transcript.text)

# Gradio ASR to output txt file (Works!)

In [11]:
import gradio as gr
from transformers import pipeline
import numpy as np

# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe_and_save(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]
    
    # Write the transcription to a file
    with open("transcription.txt", "w") as file:
        file.write(transcription)
    
    return transcription

# Create the Gradio interface
demo = gr.Interface(
    transcribe_and_save,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch(share=True)


C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://644900d8023a5d7cd1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 407, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await sel

In [12]:
#sentiment analysis with OpenAI
#preprocess txt file

with open('transcription.txt', 'r', encoding="utf-8") as file:
    text = file.read()


In [8]:
model_lst = openai.Model.list()

for i in model_lst['data']:
    print(i['id'])

gpt-4-turbo-2024-04-09
whisper-1
davinci-002
gpt-4-turbo
gpt-4-1106-preview
gpt-4-0613
dall-e-2
tts-1-hd-1106
tts-1-hd
gpt-4
gpt-4-1106-vision-preview
gpt-3.5-turbo-0125
gpt-3.5-turbo
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
tts-1
dall-e-3
gpt-3.5-turbo-0613
gpt-3.5-turbo-0301
gpt-3.5-turbo-instruct-0914
babbage-002
gpt-3.5-turbo-instruct
tts-1-1106
text-embedding-3-large
gpt-4-0125-preview
text-embedding-3-small
text-embedding-ada-002
gpt-3.5-turbo-1106
gpt-4-turbo-preview
gpt-4-vision-preview


In [27]:
#analysis

import openai
import time

# Set your OpenAI API key
openai.api_key = "sk-proj-mU2gUtjNmEIhJNoILJxNT3BlbkFJkiUSTSKHz7jJtwBuqGQ3"

# Function to perform sentiment analysis using OpenAI
def perform_sentiment_analysis(text):
    prompt = f"Please analyze the sentiment of the following text:\n{text}"
    response = openai.Completion.create(
        engine="davinci-002",
        prompt=prompt,
        temperature=0.1,
        max_tokens=128,
        n=1,
        stop=None,
        timeout=10,
    )
    sentiment = response.choices[0].text.strip().replace("The sentiment of the text is ", "").rstrip('.')
    return sentiment

# Read the text file
with open('transcription.txt', 'r', encoding="utf-8") as file:
    text = file.read()

# Perform sentiment analysis
sentiment = perform_sentiment_analysis(text)
print(f"The sentiment of the text is {sentiment}")


The sentiment of the text is I am a girl and I am 14 years old. I am from Russia. I like to read books and watch movies. I am a student and I am in the 9th grade. I am looking for a pen pal from any country. I am looking forward to hearing from you. I am waiting for your letter. Bye.

Sentiment analysis results:

Sentiment 				Calculated value 

Positive 				0.0 
Neutral 				0.0 
Negative 				0.0 

Sentiment value of the text: 0.0

The following text is also analyzed:

This is a test I feel
